In [1]:
%%capture
!pip install librosa matplotlib scipy sympy numpy --only-binary :all:

In [2]:
from IPython.display import Latex, Audio, display
import copy
import librosa.display
import numpy as np
import math
import sympy
import matplotlib.pyplot as plt
from scipy.io import wavfile
import os

In [3]:
class Signal:
    def __init__(self, values, start_index=0, sig_name=None, rate=None):
        self.values = values
        self.start_index = start_index
        self.length = len(values)
        self.end_index = self.start_index + self.length - 1
        self.sig_name = sig_name
        self.rate = rate

    def from_path(path):
        data, rate = librosa.load(path, sr=None)
        return Signal(list(data), start_index=0, rate=rate)

    def from_polynom(H0_poly, F0_poly, var):
        # Преобразуем многочлены в список коэффициентов
        H0_coeffs = sympy.Poly(H0_poly, var ** (-1)).all_coeffs()
        F0_coeffs = sympy.Poly(F0_poly, var ** (-1)).all_coeffs()
    
        # Конвертируем коэффициенты в Signal
        # Низкочастотный, получает сглаженную версию сигнала.
        H0 = Signal([float(coef) for coef in H0_coeffs])
        # низкочастотный для восстановления
        F0 = Signal([float(coef) for coef in F0_coeffs])
    
        # Генерация дополнительных фильтров H1 и F1
        # Высокочастотный, выделяет детали
        H1 = Signal([coef * (-1) ** n for n, coef in enumerate(H0.values)])
        # Высокочастотный для восстановления
        F1 = Signal([coef * (-1) ** (n + 1) for n, coef in enumerate(F0.values)])
    
        return H0, F0, H1, F1

    def mul(self, scalar):
        scaled_values = [x * scalar for x in self.values]
        return Signal(scaled_values, self.start_index, rate=self.rate)

    def add(self, other):
        new_start = min(self.start_index, other.start_index)
        new_end = max(self.end_index, other.end_index)
        new_length = new_end - new_start + 1
        result_values = [0] * new_length
        
        offset_self = self.start_index - new_start
        for i, val in enumerate(self.values):
            result_values[i + offset_self] += val

        offset_other = other.start_index - new_start
        for i, val in enumerate(other.values):
            result_values[i + offset_other] += val

        return Signal(result_values, new_start, rate=self.rate)

    def convolve(self, filter):
        signal = self.values
        kernel = filter.values
        
        matrix = []
        result_length = len(signal) + len(kernel) - 1
        result_values = [0] * result_length

        for val in kernel:
            matrix.append([val * item for item in signal])

        rows, cols = len(matrix), len(matrix[0])

        for sum_indices in range(rows + cols - 1):
            for i in range(max(0, sum_indices - cols + 1), min(sum_indices + 1, rows)):
                j = sum_indices - i
                result_values[sum_indices] += matrix[i][j]

        start_index = self.start_index + filter.start_index
        return Signal(result_values, start_index, rate=self.rate)

    def upsample(self, factor):
        upsampled_values = []
        for i, value in enumerate(self.values):
            upsampled_values.append(value)
            if i < len(self.values) - 1:
                upsampled_values.extend([0] * (factor - 1))
        return Signal(upsampled_values, self.start_index, rate=self.rate)

    def downsample(self, factor):
        downsampled_values = []
        for i in range(0, self.length, factor):
            downsampled_values.append(self.values[i])
        return Signal(downsampled_values, self.start_index, rate=self.rate)

    def __add__(self, other):
        return self.add(other)

    def Analysis(self, h0, h1):
        r0 = self.convolve(h0)
        r1 = self.convolve(h1)
        y0 = r0.downsample(2)
        y1 = r1.downsample(2)
        return y0, y1

    def Synthesis(y0,y1,f0,f1):
        t0 = y0.upsample(2)
        t1 = y1.upsample(2)
        v0 = t0.convolve(f0)
        v1 = t1.convolve(f1)
        start_index = min(v0.start_index, v1.start_index)
        v0 = Signal(v0.values, v0.start_index)
        v1 = Signal(v1.values, v1.start_index)
        result_signal = v0 + v1

        if result_signal.start_index < 0:
            while result_signal.values[0] == 0 and result_signal.start_index < 0:
                result_signal.values.pop(0)
                result_signal.start_index += 1

        original_length = y0.length + y1.length
        if len(result_signal.values) > original_length:
            result_signal.values = result_signal.values[:original_length]
        result_signal.end_index = result_signal.start_index + len(result_signal.values) - 1
        result_signal.rate = y0.rate
        return result_signal

    def factorize_polynomial(self, poly=None):
        x = sympy.symbols('x')

        if poly is None:
            polynomial = sum(
                value * x ** (index + self.start_index) for index, value in enumerate(self.values)
            )
        else:
            polynomial = poly

        factorized_polynomial = sympy.Poly(polynomial).factor_list()
        return factorized_polynomial

    def displayAudio(self):
        if self.rate is not None:
            display(Audio(np.copy(self.values), rate=self.rate))